In [8]:
import numpy as np
import pandas as pd
import yfinance as yf
import riskfolio as rp

# Helper function for Black-Litterman model
def black_litterman(mu_market, cov_matrix, P, Q, omega):
    tau = 0.025  # scaling factor typically used in Black-Litterman
    M_inverse = np.linalg.inv(tau * cov_matrix)
    
    # Calculate the intermediate matrices
    P_T = P.T
    omega_inverse = np.linalg.inv(omega)
    
    # Calculate the posterior mean (mu_bl) and posterior covariance (cov_bl)
    mu_bl = np.linalg.inv(M_inverse + P_T @ omega_inverse @ P) @ (M_inverse @ mu_market + P_T @ omega_inverse @ Q)
    cov_bl = np.linalg.inv(M_inverse + P_T @ omega_inverse @ P)
    
    return mu_bl, cov_bl

# 1. Download stock and bond data using yfinance

tickers = ['360200.KS', '449170.KS']  # Stocks (KODEX 200 ETF) and Bonds (KOSEF 단기채권)
data = yf.download(tickers, start='2022-01-01', end='2023-01-01')['Adj Close']

# Calculate daily returns
returns = data.pct_change().dropna()

# Market implied returns (Prior) - assumed returns for the assets
mu_market = returns.mean().values  # Expected returns for stocks and bonds
cov_matrix = returns.cov().values  # Covariance matrix for stocks and bonds

# 2. Define views (P) and uncertainty (omega) based on scenarios

# Define views based on scenarios (P matrix)
P = np.array([[1, 0],  # View 1: stocks expected to increase
              [1, 0],  # View 2: stocks expected to increase more
              [1, 0],  # View 3: stocks expected to increase slightly
              [1, 0]]) # View 4: stocks expected to fall

# Expected returns based on each scenario (Q vector)
Q = np.array([0.0804, 0.103, 0.02, -0.079])  # Scenario 1, 2, 3, 4 expected returns

# View uncertainty (Omega matrix)
omega = np.diag([0.005, 0.01, 0.005, 0.02])  # Uncertainty for each view

# 3. Implement Black-Litterman model using custom function

mu_bl, cov_bl = black_litterman(mu_market, cov_matrix, P, Q, omega)

# 4. Optimize portfolio

# Create portfolio object
port = rp.Portfolio(returns=returns)

# Set the Black-Litterman expected returns and covariance
port.mu = pd.Series(mu_bl, index=['360200.KS', '449170.KS'])
port.cov = pd.DataFrame(cov_bl, index=['360200.KS', '449170.KS'], columns=['360200.KS', '449170.KS'])

# Set constraints (e.g., weights sum to 1, non-negative weights)
port.constraints = {'min_weights': [0, 0], 'max_weights': [1, 1]}  # No short selling

# Optimize portfolio for maximum Sharpe ratio
weights = port.optimization(model='Classic', rm='MV', obj='Sharpe')

# Display the resulting optimal portfolio weights
print("Optimal portfolio weights based on Black-Litterman model:")
print(weights)


[*********************100%%**********************]  2 of 2 completed

Optimal portfolio weights based on Black-Litterman model:
                weights
360200.KS  2.736106e-13
449170.KS  1.000000e+00
